In [27]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc

%matplotlib inline

In [28]:
number = preprocessing.LabelEncoder()
in_file = 'titanic_data.csv'
train = pd.read_csv(in_file)
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


Step 2 convert categorical lable to numpy arrays and fill NaN values to zero

In [30]:
def convert(data):
    number = preprocessing.LabelEncoder()
    data['Sex'] = number.fit_transform(data.Sex)
    data['Name'] = number.fit_transform(data.Name)
    data['Ticket'] = number.fit_transform(data.Ticket)
    data['Embarked'] = number.fit_transform(data.Embarked)
    data['Cabin'] = number.fit_transform(data.Cabin)
    return data

train = convert(train)
train = train.fillna(0)
train.describe()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,445.000000,0.647587,23.799293,0.523008,0.381594,338.528620,32.204208,17.629630,2.529742
std,257.353842,0.486592,0.836071,257.353842,0.477990,17.596074,1.102743,0.806057,200.850657,49.693429,38.140335,0.800254
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,222.500000,0.000000,6.000000,0.000000,0.000000,158.500000,7.910400,0.000000,2.000000
50%,446.000000,0.000000,3.000000,445.000000,1.000000,24.000000,0.000000,0.000000,337.000000,14.454200,0.000000,3.000000
75%,668.500000,1.000000,3.000000,667.500000,1.000000,35.000000,1.000000,0.000000,519.500000,31.000000,0.000000,3.000000
max,891.000000,1.000000,3.000000,890.000000,1.000000,80.000000,8.000000,6.000000,680.000000,512.329200,147.000000,3.000000


Step 3 Split the dataset to train and validate

In [52]:
#ingenious wa to mark randomly 75% of rows as true
train['is_train'] = np.random.uniform(0,1,len(train)) <= 0.75
train, validate = train[train['is_train'] == True], train[train['is_train'] == False]
train = train.drop('is_train',axis=1)
y_train = train['Survived']
x_train = train.drop('Survived',axis=1)
y_train.head()
validate = validate.drop('is_train', axis=1)
y_validate = validate['Survived']
x_validate = validate.drop('Survived', axis=1)
x_validate

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
29,30,3,813,1,0.0,0,0,362,7.8958,0,3
144,145,2,32,1,18.0,0,0,118,11.5000,0,3
168,169,1,69,1,0.0,0,0,576,25.9250,0,3
205,206,3,789,0,2.0,0,1,316,10.4625,146,3
216,217,3,375,0,27.0,0,0,670,7.9250,0,3
245,246,1,539,1,44.0,2,0,92,90.0000,79,2
308,309,2,3,1,30.0,1,0,575,24.0000,0,1
329,330,1,363,0,16.0,0,1,7,57.9792,18,1
483,484,3,825,0,63.0,0,0,489,9.5875,0,3
501,502,3,133,0,21.0,0,0,433,7.7500,0,2


In [58]:
lg = LogisticRegression()
lg.fit(x_train,y_train)
survived_lg = lg.predict_proba(x_validate)
survived_lg # 2d array take the second prob value
survived_lg


array([[ 0.88571054,  0.11428946],
       [ 0.84520479,  0.15479521],
       [ 0.7034914 ,  0.2965086 ],
       [ 0.09637476,  0.90362524],
       [ 0.44163866,  0.55836134],
       [ 0.63431277,  0.36568723],
       [ 0.64344656,  0.35655344],
       [ 0.266137  ,  0.733863  ],
       [ 0.53755059,  0.46244941],
       [ 0.27732771,  0.72267229],
       [ 0.66820642,  0.33179358],
       [ 0.5879548 ,  0.4120452 ],
       [ 0.75896128,  0.24103872],
       [ 0.73182804,  0.26817196],
       [ 0.71516202,  0.28483798],
       [ 0.72193666,  0.27806334],
       [ 0.63793458,  0.36206542],
       [ 0.14071674,  0.85928326],
       [ 0.25191492,  0.74808508],
       [ 0.11835323,  0.88164677],
       [ 0.27372846,  0.72627154]])

In [59]:
fpr, tpr, _ = roc_curve(y_validate,survived_lg[:,1])
roc_auc = auc(fpr,tpr)
roc_auc

0.63265306122448983

In [61]:
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train,y_train)
survived_rf = rf.predict_proba(x_validate)

In [62]:
fpr, tpr, _ = roc_curve(y_validate,survived_rf[:,1])
roc_auc = auc(fpr,tpr)
roc_auc

0.69387755102040816